 Pipelineを用いたベースラインモデルの簡単な作り方
 
Kaggleの Housing Prices Competitions for Kaggle learn users のデータを使用して
Pipeline の使い方を学んでいきます。Pipeline を使えば複雑なコードの可読性を高めることができるので、
モデルの変更やimputerの変更が容易になり、前処理の生産性を上げることができます。

まずは必要なパッケージを読み込みます。

In [1]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

次にデータを読み込みます。ここではId列で添字付けられた各住宅に関する80の特徴量（床面積、車庫の有無、築年数など）
が格納されたテーブルデータを用います。このコンペではこれらの特徴量を用いて各住宅の住宅価格を推定することが目的となります。
使用するデータのshapeは訓練データ(1460 x 81)、テストデータ(1459 x 80)となります。target変数は住宅価格である SalePrice です。
訓練データをtrain、テストデータをtestとして読み込みます。


In [2]:
import pandas as pd

train = pd.read_csv('~/Downloads/kaggle_house/train.csv')
test = pd.read_csv('~/Downloads/kaggle_house/test.csv')


In [3]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [5]:
# yに目的変数であるSalePriceを、Xに特徴量を格納します。

y = train.SalePrice
X = train.drop(['SalePrice'], axis=1)


In [6]:
# カテゴリ変数と数値変数のコラム名を取得します。

cat_cols = [cname for cname in X.columns if X[cname].dtype == 'object']
num_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]


In [7]:
# ここでpipelineの前処理を定義していきます。数値変数の欠損値はSimpleImputerを用いて中央値で補完し、\
# カテゴリ変数の欠損値補完には最頻値を、エンコーディングにはOneHotEncoderを用いることにします。pipelineを用いれば\
# カテゴリ変数のimputerとencoderをシンプルに記述出来ることがわかります。

num_imputer = SimpleImputer(strategy='median')
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# preprocessorに数値変数とカテゴリ変数に対して施した前処理の定義を渡します。この短いコードで前処理で何をしたのか\
# 一目でわかるようになります。

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_imputer, num_cols),
        ('cat', cat_transformer, cat_cols)
    ]
)


In [8]:
# モデルの定義です。今回はxgboostを使用することにします。もちろんlightgbmでも構いません。

from xgboost import XGBRegressor
xgb_model = XGBRegressor(n_estimators=1000, learning_rate=0.04,
                         n_jobs=4)

In [9]:
# pipeline に構築した前処理とモデルを格納します。これでベースラインモデルの完成です。

my_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', xgb_model)
])


In [10]:
# 試しにスコア(mean absolute error)を見てみます。pipelineを用いればシンプルに\
# クロスバリデーションのコードが記述出来ることが見て取れます。

from sklearn.model_selection import cross_val_score

scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print(scores)
print(scores.mean())

[15322.61900685 16327.39784889 16433.05316246 14277.85192369
 16690.62607021]
15810.309602418663
